In [1]:
import logging
import hydra
import pandas as pd

from collections import defaultdict
from tqdm import tqdm 

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colormaps
import matplotlib.cm as cm
import matplotlib as mpl
from matplotlib.patches import Ellipse

from erc import drawing_ellipse, split_df_by_gender


logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

with hydra.initialize(version_base=None, config_path="./config"):
    cfg = hydra.compose(config_name="config", overrides={"dataset._target_=erc.datasets.KEMDy19Dataset"})


# select 1-fold 
cfg.dataset.mode = "train"
train_dataset = hydra.utils.instantiate(cfg.dataset)

cfg.dataset.mode = "valid"
cfg.dataset.validation_fold = 0
valid_dataset = hydra.utils.instantiate(cfg.dataset)

INFO:erc.utils:Instantiate KEMDy19 Dataset


INFO:erc.utils:Instantiate KEMDy19 Dataset


In [56]:
path = "./data/KEMDy19/wav/Session01/Sess01_impro01/Sess01_impro01_F001.wav"
import torchaudio 



waveform, sample_rate = torchaudio.load(path)

# waveform = waveform.type(torch.FloatTensor)

In [58]:
type(waveform)

torch.Tensor

In [2]:
# Pre-training Scheme ... 

from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size= 16)

In [7]:
# from transformers import Wav2Vec2Processor, Wav2Vec2Model
# extractor = Wav2Vec2Model.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")

Some weights of the model checkpoint at kresnik/wav2vec2-large-xlsr-korean were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (1): Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (2): Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (3): Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
    

In [50]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2ForSequenceClassification

pretrained_model = Wav2Vec2ForSequenceClassification.from_pretrained(
    # "wav2vec2-xls-r-300m-korean",
    "w11wo/wav2vec2-xls-r-300m-korean",
    num_labels=7
    )


Some weights of the model checkpoint at w11wo/wav2vec2-xls-r-300m-korean were not used when initializing Wav2Vec2ForSequenceClassification: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at w11wo/wav2vec2-xls-r-300m-korean and are newly initialized: ['projector.bias', 'projector.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for p

In [6]:
# def wav2processor()

# train_dataset.map()

In [3]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
from transformers import AdamW
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
from jiwer import wer # wer metircs 
from transformers import Wav2Vec2Processor

class EmotionClassfierWithAudio(nn.Module):
    def __init__(self,pretrained_model, n_classes=7):
        super().__init__()
        self.pretrianed_model = pretrained_model
        # for p in self.pretrianed_model.parameters():
            # p.requires_grad = False

        self.fc = nn.Linear(1025,n_classes)

    def forward(self, x):
        x = F.adaptive_avg_pool3d(self.pretrianed_model(x).logits , axis=1).squeeze()# mean of 355 // logit output shape: B, Seq(355), 1025
        x = self.fc(x)
        return x

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/hoesungryu/.conda/envs/etri-erc/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [37]:
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
pretrained_model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
model = EmotionClassfierWithAudio(pretrained_model).to("cuda")
criterion = nn.MultiLabelSoftMarginLoss()
optimizer = AdamW(model.parameters(), lr = 1e-5,  eps = 1e-8)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/hoesungryu/.conda/envs/etri-erc/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [52]:
import torch 
total_loss = 0
train_acc_sum = 0
train_loss = []

for batch in train_loader: 
    optimizer.zero_grad()
    print(batch.keys())
    input_values = batch['wav'].type(torch.FloatTensor)
    attention_mask = batch['wav_mask']
    print(input_values)
    print(attention_mask)
    
    # inputs = processor(batch["wav"], sampling_rate=16000, return_tensors="pt")
    # labels = batch['emotion'].to("cuda")
    
    # input_values = inputs.input_values.type(torch.FloatTensor).squeeze().to("cuda")
    # print(input_values.shape)
    break
    

dict_keys(['sampling_rate', 'wav', 'wav_mask', 'txt', 'txt_mask', 'emotion', 'valence', 'arousal', 'gender'])
tensor([[   15,     5,    10,  ...,     0,     0,     0],
        [  -73,   -70,   -49,  ...,     0,     0,     0],
        [   -4,     8,    23,  ...,     0,     0,     0],
        ...,
        [ -880,  -862,  -850,  ...,     0,     0,     0],
        [ -137,  -121,   -67,  ...,     0,     0,     0],
        [   -2,    -9,     7,  ..., 32701, 32767, 32655]], dtype=torch.int16)
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])


In [45]:
# pretrained_model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
pretrained_model(input_values).logits.shape

torch.Size([16, 624, 1205])

In [39]:
model(input_values)

RuntimeError: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [1, 1, 16, 200000]

tensor([[   15,     5,    10,  ...,     0,     0,     0],
        [  -73,   -70,   -49,  ...,     0,     0,     0],
        [   -4,     8,    23,  ...,     0,     0,     0],
        ...,
        [ -880,  -862,  -850,  ...,     0,     0,     0],
        [ -137,  -121,   -67,  ...,     0,     0,     0],
        [   -2,    -9,     7,  ..., 32701, 32767, 32655]], dtype=torch.int16)

In [20]:
a = extractor(batch["wav"].type(torch.FloatTensor))

In [24]:
a.extract_features.shape

torch.Size([16, 624, 512])

In [36]:
a.extract_features[:,-1].shape

torch.Size([16, 512])

In [34]:
F.adaptive_avg_pool1d(a.extract_features, output_size = 7).shape

torch.Size([16, 624, 7])

In [ ]:

a.extract_features.

In [5]:
model(input_values)

RuntimeError: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [1, 1, 16, 200000]

In [155]:
inputs.input_values.shape

torch.Size([1, 1, 130698])

In [ ]:

    outputs = model(input_values)

    loss = criterion(outputs, labels)
    total_loss += loss.item()
    train_loss.append(total_loss/(step+1))

    loss.backward()
    optimizer.step()

avg_train_loss = total_loss / len(train_loader)
print(f'  Average training loss: {avg_train_loss:.2f}')


In [124]:
# !pip install jiwer
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
# import soundfile as sf
import torch
from jiwer import wer # wer metircs 
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")

batch = train_dataset[2]

inputs = processor(batch["wav"], sampling_rate=16000, return_tensors="pt", padding="longest")
model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to('cuda')

input_values = inputs.input_values
print(input_values.shape)
with torch.no_grad():
    logits = model(input_values.type(torch.FloatTensor).to("cuda")).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
print(batch['txt'])
print(transcription)
print("WER:", wer(batch['txt'], transcription))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


torch.Size([1, 107290])
['아이 씨 그러니까 나 말렸어야지. 술 먹어서 운전 안 한다고 했잖아. b/\n']
['아이그럴게다 말려진 이 스무 성장 한라이자']
WER: 1.0


In [125]:
logits.shape

torch.Size([1, 335, 1205])

In [84]:
logits[:,-1].shape

torch.Size([1, 1205])

In [86]:
logits.shape

torch.Size([1, 335, 1205])

In [88]:
logits.mean(axis=1).shape

torch.Size([1, 1205])

tensor([[-14.0799,  -4.3673,  -9.6379,  ...,  -9.3673, -15.0763,  21.4111]],
       device='cuda:0')

In [91]:
logits[:,-1]

tensor([[-13.9847,  -4.8777, -10.1794,  ..., -13.5519, -14.5768,  15.8865]],
       device='cuda:0')

In [89]:
logits

tensor([[[-19.6886,  -5.0163, -13.3191,  ..., -13.3676, -16.0853,  26.1782],
         [-18.5488,  -4.6266, -12.9144,  ..., -13.5130, -15.9508,  26.7478],
         [-18.4327,  -5.4421, -12.1068,  ..., -13.6475, -17.1102,  27.5597],
         ...,
         [-13.1175,  -4.8050,  -3.3759,  ...,  -4.1841, -10.5492,  14.7960],
         [-13.8290,  -2.7419,  -4.8392,  ...,  -9.3199, -12.3258,  17.4055],
         [-13.9847,  -4.8777, -10.1794,  ..., -13.5519, -14.5768,  15.8865]]],
       device='cuda:0')

In [87]:
logits

tensor([[[-19.6886,  -5.0163, -13.3191,  ..., -13.3676, -16.0853,  26.1782],
         [-18.5488,  -4.6266, -12.9144,  ..., -13.5130, -15.9508,  26.7478],
         [-18.4327,  -5.4421, -12.1068,  ..., -13.6475, -17.1102,  27.5597],
         ...,
         [-13.1175,  -4.8050,  -3.3759,  ...,  -4.1841, -10.5492,  14.7960],
         [-13.8290,  -2.7419,  -4.8392,  ...,  -9.3199, -12.3258,  17.4055],
         [-13.9847,  -4.8777, -10.1794,  ..., -13.5519, -14.5768,  15.8865]]],
       device='cuda:0')

In [80]:
model(input_values.type(torch.FloatTensor).to("cuda"))

AttributeError: 'CausalLMOutput' object has no attribute 'extract_features'

In [79]:
predicted_ids

tensor([[1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204,
         1204, 1204, 1204, 1204,  886, 1204, 1204, 1204, 1204, 1204, 1204, 1204,
         1204,  127, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204,
         1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204,
         1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204,
         1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204,
         1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204,
         1204, 1204,  469, 1204, 1204, 1204, 1052, 1204, 1204, 1204, 1204, 1204,
          980, 1204, 1204, 1204,  962, 1204, 1204,  859, 1204,  516, 1204, 1204,
         1204, 1204, 1204,  853, 1204, 1204, 1204, 1204, 1204, 1204, 1204,  702,
         1204,  859,  859, 1204,  127, 1204, 1204, 1204, 1204, 1204, 1204, 1204,
         1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204, 1204,
         1204,  859,  859,  